<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению. Сессия № 2
</center>
Автор материала: Юрий Исаков и Юрий Кашницкий. Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Тема 4. Линейные модели классификации и регрессии
## <center>  Практика. Идентификация пользователя с помощью логистической регрессии

Тут мы воспроизведем парочку бенчмарков нашего соревнования и вдохновимся побить третий бенчмарк, а также остальных участников. Веб-формы для отправки ответов тут не будет, ориентир – [leaderboard](https://www.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/leaderboard) соревнования.

In [1]:
import pickle

import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, hstack
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm_notebook

%matplotlib inline
import seaborn as sns
from matplotlib import pyplot as plt

### 1. Загрузка и преобразование данных
Зарегистрируйтесь на [Kaggle](www.kaggle.com), если вы не сделали этого раньше, зайдите на [страницу](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) соревнования и скачайте данные. Первым делом загрузим обучающую и тестовую выборки и посмотрим на данные.

In [2]:
# загрузим обучающую и тестовую выборки
train_df = pd.read_csv("../../data/train_sessions.csv", index_col="session_id")
test_df = pd.read_csv("../../data/test_sessions.csv", index_col="session_id")

# приведем колонки time1, ..., time10 к временному формату
times = ["time%s" % i for i in range(1, 11)]
train_df[times] = train_df[times].apply(pd.to_datetime)
test_df[times] = test_df[times].apply(pd.to_datetime)

# отсортируем данные по времени
train_df = train_df.sort_values(by="time1")

# посмотрим на заголовок обучающей выборки
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
21669,56,2013-01-12 08:05:57,55.0,2013-01-12 08:05:57,NaN,NaT,NaN,NaT,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
54843,56,2013-01-12 08:37:23,55.0,2013-01-12 08:37:23,56.0,2013-01-12 09:07:07,55.0,2013-01-12 09:07:09,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
77292,946,2013-01-12 08:50:13,946.0,2013-01-12 08:50:14,951.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:16,...,2013-01-12 08:50:16,948.0,2013-01-12 08:50:16,784.0,2013-01-12 08:50:16,949.0,2013-01-12 08:50:17,946.0,2013-01-12 08:50:17,0
114021,945,2013-01-12 08:50:17,948.0,2013-01-12 08:50:17,949.0,2013-01-12 08:50:18,948.0,2013-01-12 08:50:18,945.0,2013-01-12 08:50:18,...,2013-01-12 08:50:18,947.0,2013-01-12 08:50:19,945.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:20,0
146670,947,2013-01-12 08:50:20,950.0,2013-01-12 08:50:20,948.0,2013-01-12 08:50:20,947.0,2013-01-12 08:50:21,950.0,2013-01-12 08:50:21,...,2013-01-12 08:50:21,946.0,2013-01-12 08:50:21,951.0,2013-01-12 08:50:22,946.0,2013-01-12 08:50:22,947.0,2013-01-12 08:50:22,0


В обучающей выборке содержатся следующие признаки:
    - site1 – индекс первого посещенного сайта в сессии
    - time1 – время посещения первого сайта в сессии
    - ...
    - site10 – индекс 10-го посещенного сайта в сессии
    - time10 – время посещения 10-го сайта в сессии
    - target – целевая переменная, 1 для сессий Элис, 0 для сессий других пользователей
    
Сессии пользователей выделены таким образом, что они не могут быть длиннее получаса или 10 сайтов. То есть сессия считается оконченной либо когда пользователь посетил 10 сайтов подряд либо когда сессия заняла по времени более 30 минут.

В таблице встречаются пропущенные значения, это значит, что сессия состоит менее, чем из 10 сайтов. Заменим пропущенные значения нулями и приведем признаки к целому типу. Также загрузим словарь сайтов и посмотрим, как он выглядит:

In [3]:
train_df[train_df.target == 1].head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
251175,270,2013-02-12 16:25:10,270.0,2013-02-12 16:25:11,270.0,2013-02-12 16:32:10,21.0,2013-02-12 16:32:11,21.0,2013-02-12 16:32:24,...,2013-02-12 16:32:25,21.0,2013-02-12 16:32:25,7832.0,2013-02-12 16:32:26,30.0,2013-02-12 16:32:27,7832.0,2013-02-12 16:32:27,1
196388,29,2013-02-12 16:32:27,7832.0,2013-02-12 16:32:28,37.0,2013-02-12 16:32:29,7832.0,2013-02-12 16:32:34,7832.0,2013-02-12 16:32:35,...,2013-02-12 16:32:35,7832.0,2013-02-12 16:32:42,29.0,2013-02-12 16:32:42,7832.0,2013-02-12 16:32:51,7832.0,2013-02-12 16:32:53,1
172448,29,2013-02-12 16:32:53,7832.0,2013-02-12 16:33:11,7832.0,2013-02-12 16:33:12,29.0,2013-02-12 16:33:13,37.0,2013-02-12 16:33:15,...,2013-02-12 16:33:24,29.0,2013-02-12 16:33:24,7832.0,2013-02-12 16:33:33,29.0,2013-02-12 16:33:34,270.0,2013-02-12 16:33:46,1
70129,167,2013-02-12 16:33:50,167.0,2013-02-12 16:33:51,1515.0,2013-02-12 16:33:52,167.0,2013-02-12 16:33:52,37.0,2013-02-12 16:33:52,...,2013-02-12 16:33:52,855.0,2013-02-12 16:33:52,1515.0,2013-02-12 16:33:53,855.0,2013-02-12 16:33:53,1514.0,2013-02-12 16:33:53,1
206254,1520,2013-02-12 16:33:55,1522.0,2013-02-12 16:33:56,1522.0,2013-02-12 16:34:01,1515.0,2013-02-12 16:34:12,1515.0,2013-02-12 16:34:13,...,2013-02-12 16:34:24,1514.0,2013-02-12 16:34:24,1515.0,2013-02-12 16:34:24,1520.0,2013-02-12 16:34:25,1521.0,2013-02-12 16:34:25,1


In [4]:
# приведем колонки site1, ..., site10 к целочисленному формату и заменим пропуски нулями
sites = ["site%s" % i for i in range(1, 11)]
train_df[sites] = train_df[sites].fillna(0).astype("int")
test_df[sites] = test_df[sites].fillna(0).astype("int")

# загрузим словарик сайтов
with open(r"../../data/site_dic.pkl", "rb") as input_file:
    site_dict = pickle.load(input_file)

# датафрейм словарика сайтов
sites_dict_df = pd.DataFrame(
    list(site_dict.keys()), index=list(site_dict.values()), columns=["site"]
)
print(u"всего сайтов:", sites_dict_df.shape[0])
sites_dict_df.head()

всего сайтов: 48371


,site
25075,www.abmecatronique.com
13997,groups.live.com
42436,majeureliguefootball.wordpress.com
30911,cdt46.media.tourinsoft.eu
8104,www.hdwallpapers.eu


Выделим целевую переменную и объединим выборки, чтобы вместе привести их к разреженному формату.

In [5]:
# наша целевая переменная
y_train = train_df["target"]

# объединенная таблица исходных данных
full_df = pd.concat([train_df.drop("target", axis=1), test_df])

# индекс, по которому будем отделять обучающую выборку от тестовой
idx_split = train_df.shape[0]

In [6]:
full_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,site6,time6,site7,time7,site8,time8,site9,time9,site10,time10
session_id,,,,,,,,,,,,,,,,,,,,
21669,56,2013-01-12 08:05:57,55,2013-01-12 08:05:57,0,NaT,0,NaT,0,NaT,0,NaT,0,NaT,0,NaT,0,NaT,0,NaT
54843,56,2013-01-12 08:37:23,55,2013-01-12 08:37:23,56,2013-01-12 09:07:07,55,2013-01-12 09:07:09,0,NaT,0,NaT,0,NaT,0,NaT,0,NaT,0,NaT
77292,946,2013-01-12 08:50:13,946,2013-01-12 08:50:14,951,2013-01-12 08:50:15,946,2013-01-12 08:50:15,946,2013-01-12 08:50:16,945,2013-01-12 08:50:16,948,2013-01-12 08:50:16,784,2013-01-12 08:50:16,949,2013-01-12 08:50:17,946,2013-01-12 08:50:17
114021,945,2013-01-12 08:50:17,948,2013-01-12 08:50:17,949,2013-01-12 08:50:18,948,2013-01-12 08:50:18,945,2013-01-12 08:50:18,946,2013-01-12 08:50:18,947,2013-01-12 08:50:19,945,2013-01-12 08:50:19,946,2013-01-12 08:50:19,946,2013-01-12 08:50:20
146670,947,2013-01-12 08:50:20,950,2013-01-12 08:50:20,948,2013-01-12 08:50:20,947,2013-01-12 08:50:21,950,2013-01-12 08:50:21,952,2013-01-12 08:50:21,946,2013-01-12 08:50:21,951,2013-01-12 08:50:22,946,2013-01-12 08:50:22,947,2013-01-12 08:50:22


In [7]:
y_train.shape

(253561,)

Для самой первой модели будем использовать только посещенные сайты в сессии (но не будем обращать внимание на временные признаки). За таким выбором данных для модели стоит такая идея:  *у Элис есть свои излюбленные сайты, и чем чаще вы видим эти сайты в сессии, тем выше вероятность, что это сессия Элис и наоборот.*

Подготовим данные, из всей таблицы выберем только признаки `site1, site2, ... , site10`. Напомним, что пропущенные значения заменены нулем. Вот как выглядят первые строки таблицы:

In [8]:
# табличка с индексами посещенных сайтов в сессии
full_sites = full_df[sites]
full_sites.head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
21669,56,55,0,0,0,0,0,0,0,0
54843,56,55,56,55,0,0,0,0,0,0
77292,946,946,951,946,946,945,948,784,949,946
114021,945,948,949,948,945,946,947,945,946,946
146670,947,950,948,947,950,952,946,951,946,947


Сессии представляют собой последовательность индексов сайтов и данные в таком виде неудобны для линейных методов. В соответствии с нашей гипотезой (у Элис есть излюбленные сайты) надо преобразовать эту таблицу таким образом, чтобы каждому возможному сайту соответствовал свой отдельный признак (колонка), а его значение равнялось бы количеству посещений этого сайта в сессии. Это делается в две строчки:

In [9]:
from scipy.sparse import csr_matrix

In [10]:
csr_matrix?

Init signature: csr_matrix(arg1, shape=None, dtype=None, copy=False, *, maxprint=None)
Docstring:     
Compressed Sparse Row matrix.

This can be instantiated in several ways:
    csr_matrix(D)
        where D is a 2-D ndarray

    csr_matrix(S)
        with another sparse array or matrix S (equivalent to S.tocsr())

    csr_matrix((M, N), [dtype])
        to construct an empty matrix with shape (M, N)
        dtype is optional, defaulting to dtype='d'.

    csr_matrix((data, (row_ind, col_ind)), [shape=(M, N)])
        where ``data``, ``row_ind`` and ``col_ind`` satisfy the
        relationship ``a[row_ind[k], col_ind[k]] = data[k]``.

    csr_matrix((data, indices, indptr), [shape=(M, N)])
        is the standard CSR representation where the column indices for
        row i are stored in ``indices[indptr[i]:indptr[i+1]]`` and their
        corresponding values are stored in ``data[indptr[i]:indptr[i+1]]``.
        If the shape parameter is not supplied, the matrix dimensions
        

In [11]:
# последовательность с индексами
sites_flatten = full_sites.values.flatten()

# искомая матрица
full_sites_sparse = csr_matrix(
    (
        [1] * sites_flatten.shape[0],
        sites_flatten,
        range(0, sites_flatten.shape[0] + 10, 10),
    )
)[:, 1:]

In [12]:
sites_flatten

array([  56,   55,    0, ..., 1098, 1098, 1098], shape=(3363580,))

Еще один плюс использования разреженных матриц в том, что для них имеются специальные реализации как матричных операций, так и алгоритмов машинного обучения, что подчас позволяет ощутимо ускорить операции за счет особенностей структуры данных. Это касается и логистической регрессии. Вот теперь у нас все готово для построения нашей первой модели.

### 2. Построение первой модели

Итак, у нас есть алгоритм и данные для него, построим нашу первую модель, воспользовавшись релизацией [логистической регрессии](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) из пакета `sklearn` с параметрами по умолчанию. Первые 90% данных будем использовать для обучения (обучающая выборка отсортирована по времени), а оставшиеся 10% для проверки качества (validation). 

**Напишите простую функцию, которая будет возвращать качество модели на отложенной выборке, и обучите наш первый классификатор**.

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 253561 entries, 21669 to 204762
Data columns (total 21 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   site1   253561 non-null  int64         
 1   time1   253561 non-null  datetime64[ns]
 2   site2   253561 non-null  int64         
 3   time2   250098 non-null  datetime64[ns]
 4   site3   253561 non-null  int64         
 5   time3   246919 non-null  datetime64[ns]
 6   site4   253561 non-null  int64         
 7   time4   244321 non-null  datetime64[ns]
 8   site5   253561 non-null  int64         
 9   time5   241829 non-null  datetime64[ns]
 10  site6   253561 non-null  int64         
 11  time6   239495 non-null  datetime64[ns]
 12  site7   253561 non-null  int64         
 13  time7   237297 non-null  datetime64[ns]
 14  site8   253561 non-null  int64         
 15  time8   235224 non-null  datetime64[ns]
 16  site9   253561 non-null  int64         
 17  time9   233084 non-null  datet

In [15]:
y_train.shape

(253561,)

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

def get_auc_lr_valid(X, y, C=1.0, ratio=0.9, seed=17):
    """
    X, y — выборка
    ratio — в каком отношении поделить выборку (доля train)
    C, seed — коэффициент регуляризации и random_state для логистической регрессии
    """
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=ratio, random_state=seed)

    model = LogisticRegression(C=C, random_state=seed)
    model.fit(X_train, y_train)

    y_pred = model.predict_proba(X_test)[:, 1]

    auc_score = roc_auc_score(y_test, y_pred)
    return auc_score

**Посмотрите, какой получился ROC AUC на отложенной выборке.**

In [17]:
# Ваш код здесь
auc_score = get_auc_lr_valid(full_sites_sparse[:idx_split], y_train)
print(f'AUC-ROC на валидационной выборке: {auc_score}')

AUC-ROC на валидационной выборке: 0.9644876297951692


Будем считать эту модель нашей первой отправной точкой (baseline). Для построения модели для прогноза на тестовой выборке **необходимо обучить модель заново уже на всей обучающей выборке** (пока наша модель обучалась лишь на части данных), что повысит ее обобщающую способность:

In [18]:
# функция для записи прогнозов в файл
def write_to_submission_file(
    predicted_labels, out_file, target="target", index_label="session_id"
):
    predicted_df = pd.DataFrame(
        predicted_labels,
        index=np.arange(1, predicted_labels.shape[0] + 1),
        columns=[target],
    )
    predicted_df.to_csv(out_file, index_label=index_label)

**Обучите модель на всей выборке, сделайте прогноз для тестовой выборки и сделайте посылку в соревновании**.

In [19]:
# Ваш код здесь
model = LogisticRegression(C=1.0, random_state=17)
model.fit(full_sites_sparse[:idx_split], y_train)

y_pred = model.predict_proba(full_sites_sparse[idx_split:])[:, 1]

write_to_submission_file(y_pred, 'submission.csv')

Если вы выполните эти действия и загрузите ответ на [странице](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) соревнования, то воспроизведете первый бенчмарк "Logit".

### 3. Улучшение модели, построение новых признаков

Создайте такой признак, который будет представлять собой число вида ГГГГММ от той даты, когда проходила сессия, например 201407 -- 2014 год и 7 месяц. Таким образом, мы будем учитывать помесячный [линейный тренд](http://people.duke.edu/~rnau/411trend.htm) за весь период предоставленных данных.

In [20]:
# Ваш код здесь
full_df["start_month"] = full_df["time1"].dt.year * 100 + full_df["time1"].dt.month
full_df["start_month"].head()

session_id
21669     201301
54843     201301
77292     201301
114021    201301
146670    201301
Name: start_month, dtype: int32

Добавьте новый признак, предварительно отмасштабировав его с помощью `StandardScaler`, и снова посчитайте ROC AUC на отложенной выборке.

In [21]:
# Ваш код здесь
scaler = StandardScaler()
full_df["start_month"] = scaler.fit_transform(full_df[["start_month"]])

In [22]:
# full_df = full_df.drop("year_month", axis=1)
# full_df.head()

In [23]:
year_month_sparse = csr_matrix(full_df[["start_month"]].values)
full_sites_sparse_with_month = hstack([full_sites_sparse, year_month_sparse])

In [24]:
X_train = full_sites_sparse_with_month[:idx_split]
X_test = full_sites_sparse_with_month[idx_split:]
auc_with_month = get_auc_lr_valid(X_train, y_train)
print(f'ROC AUC с учетом year_month: {auc_with_month}')

ROC AUC с учетом year_month: 0.9695244468367784


**Добавьте два новых признака: start_hour и morning.**

Признак `start_hour` – это час в который началась сессия (от 0 до 23), а бинарный признак `morning` равен 1, если сессия началась утром и 0, если сессия началась позже (будем считать, что утро это если `start_hour равен` 11 или меньше).

**Посчитйте ROC AUC на отложенной выборке для выборки с:**
- сайтами, `start_month` и `start_hour`
- сайтами, `start_month` и `morning`
- сайтами, `start_month`, `start_hour` и `morning`

In [25]:
# Ваш код здесь
full_df['start_hour'] = full_df['time1'].dt.hour
full_df['morning'] = full_df['start_hour'].apply(lambda x: 1 if x <= 11 else 0)
full_df.head(2)

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time7,site8,time8,site9,time9,site10,time10,start_month,start_hour,morning
session_id,,,,,,,,,,,,,,,,,,,,,
21669,56,2013-01-12 08:05:57,55,2013-01-12 08:05:57,0,NaT,0,NaT,0,NaT,...,NaT,0,NaT,0,NaT,0,NaT,-2.079076,8,1
54843,56,2013-01-12 08:37:23,55,2013-01-12 08:37:23,56,2013-01-12 09:07:07,55,2013-01-12 09:07:09,0,NaT,...,NaT,0,NaT,0,NaT,0,NaT,-2.079076,8,1


In [26]:
full_df['start_hour'] = scaler.fit_transform(full_df[['start_hour']])

In [27]:
start_hour_sparse = csr_matrix(full_df[['start_hour']].values)
morning_sparse = csr_matrix(full_df[['morning']].values)

In [28]:
# сайтами, start_month и start_hour
X1 = hstack([full_sites_sparse, year_month_sparse, start_hour_sparse])
X1_train = X1[:idx_split]
X1_test = X1[idx_split:]
auc1 = get_auc_lr_valid(X1_train, y_train)
print(f"ROC AUC (сайты + year_month + start_hour): {auc1:.5f}")

ROC AUC (сайты + year_month + start_hour): 0.97971


In [29]:
# сайтами, start_month и morning
X2 = hstack([full_sites_sparse, year_month_sparse, morning_sparse])
X2_train = X2[:idx_split]
X2_test = X2[idx_split:]
auc2 = get_auc_lr_valid(X2_train, y_train)
print(f"ROC AUC (сайты + year_month + morning): {auc2:.5f}")

ROC AUC (сайты + year_month + morning): 0.97840


In [30]:
# сайтами, start_month, start_hour и morning
X3 = hstack([full_sites_sparse, year_month_sparse, start_hour_sparse, morning_sparse])
X3_train = X3[:idx_split]
X3_test = X3[idx_split:]
auc3 = get_auc_lr_valid(X3_train, y_train)
print(f"ROC AUC (сайты + year_month + start_hour + morning): {auc3:.5f}")

ROC AUC (сайты + year_month + start_hour + morning): 0.98255


In [31]:
X_full_sparse = hstack([full_sites_sparse, year_month_sparse, start_hour_sparse, morning_sparse])

In [32]:
model = LogisticRegression(C=1.0, random_state=17)
model.fit(X_full_sparse[:idx_split], y_train)

y_pred = model.predict_proba(X_full_sparse[idx_split:])[:, 1]

write_to_submission_file(y_pred, 'submission.csv')

### 4. Подбор коэффицициента регуляризации

Итак, мы ввели признаки, которые улучшают качество нашей модели по сравнению с первым бейслайном. Можем ли мы добиться большего значения метрики? После того, как мы сформировали обучающую и тестовую выборки, почти всегда имеет смысл подобрать оптимальные гиперпараметры -- характеристики модели, которые не изменяются во время обучения. Например, на 3 неделе вы проходили решающие деревья, глубина дерева это гиперпараметр, а признак, по которому происходит ветвление и его значение -- нет. В используемой нами логистической регрессии веса каждого признака изменяются и во время обучения находится их оптимальные значения, а коэффициент регуляризации остается постоянным. Это тот гиперпараметр, который мы сейчас будем оптимизировать.

Посчитайте качество на отложенной выборке с коэффициентом регуляризации, который по умолчанию `C=1`:

In [33]:
# Ваш код здесь
auc_defolt_C = get_auc_lr_valid(X_full_sparse[:idx_split], y_train, C=1)
print(f'ROC AUC при C=1: {auc_defolt_C}')

ROC AUC при C=1: 0.9825512635502561


Постараемся побить этот результат за счет оптимизации коэффициента регуляризации. Возьмем набор возможных значений C и для каждого из них посчитаем значение метрики на отложенной выборке.

Найдите `C` из `np.logspace(-3, 1, 10)`, при котором ROC AUC на отложенной выборке максимален. 

In [39]:
# Ваш код здесь
C_values = np.logspace(-3, 1, 10)
best_C = None
best_auc = 0

for C in C_values:
    auc_score = get_auc_lr_valid(X_full_sparse[:idx_split], y_train, C=C)
    print(f'ROC AUC при C={C:.5f}: {auc_score:.5f}')
    if auc_score > best_auc:
        best_auc = auc_score
        best_C = C

print(f"\nЛучшее C: {best_C:.5f}, наилучший ROC AUC: {best_auc:.5f}")

ROC AUC при C=0.00100: 0.91601
ROC AUC при C=0.00278: 0.93653
ROC AUC при C=0.00774: 0.95287
ROC AUC при C=0.02154: 0.96502
ROC AUC при C=0.05995: 0.97343
ROC AUC при C=0.16681: 0.97808
ROC AUC при C=0.46416: 0.98167
ROC AUC при C=1.29155: 0.98193
ROC AUC при C=3.59381: 0.98274
ROC AUC при C=10.00000: 0.98272

Лучшее C: 3.59381, наилучший ROC AUC: 0.98274


Наконец, обучите модель с найденным оптимальным значением коэффициента регуляризации и с построенными признаками `start_hour`, `start_month` и `morning`. Если вы все сделали правильно и загрузите это решение, то повторите второй бенчмарк соревнования.

In [41]:
best_C

np.float64(3.593813663804626)

In [42]:
# Ваш код здесь
model = LogisticRegression(C=best_C, random_state=17)
model.fit(X_full_sparse[:idx_split], y_train)

y_pred = model.predict_proba(X_full_sparse[idx_split:])[:, 1]

write_to_submission_file(y_pred, 'submission.csv')

In [46]:
# Ваш код здесь
C_values = np.logspace(-3, 1, 10)

for C in C_values:
    model = LogisticRegression(C=C, random_state=17)
    model.fit(X_full_sparse[:idx_split], y_train)
    
    y_pred = model.predict_proba(X_full_sparse[idx_split:])[:, 1]
    
    write_to_submission_file(y_pred, f'submission{C:.2f}.csv')
    